In [4]:
#!/usr/bin/python3
# -*-coding:utf-8
'''
Created on Fri Dec 1 22:22:35 2017

@author: Ray

'''
import time
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import utils # written by author
from glob import glob
from datetime import datetime, timedelta
import multiprocessing as mp
import gc # for automatic releasing memory



In [33]:
def drop_first_columns(x):
    return x.tail(n = x.shape[0] -1)
def within_n_days(x, T, n = 7):
	# n = 7, 14, 30, 60, 90,
	##################################################
	# Filtering accroding to w
	##################################################
	if T == 0:
		# w = 0:使用3月之前的資料當作history
		prediction_deadline = datetime.strptime('2017-03-01', '%Y-%m-%d')
	elif T == 1:
		# w = 1:使用2月之前的資料當作history
		prediction_deadline = datetime.strptime('2017-02-01', '%Y-%m-%d')
	elif T == 2:
		# w = 2:使用1月之前的資料當作history
		prediction_deadline = datetime.strptime('2017-01-01', '%Y-%m-%d')
	elif T == -1:
		# w = -1:使用4月之前的資料當作history
		prediction_deadline = datetime.strptime('2017-04-01', '%Y-%m-%d')
	check_points = prediction_deadline - timedelta(n) # 往prediction_deadline前n天內
	x = x[ (x.date >= check_points )]
	return x

df = df.groupby('msno').apply(drop_first_columns) # 每個user第一欄不用
df.reset_index(drop = True, inplace = True)
#df.reset_index()

In [37]:
df = pd.read_csv('small_df_for_developing_and_debugging.csv', parse_dates = ['date'])
df.head()

,msno,date
0,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2016-11-16
1,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2016-11-17
2,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2016-11-18
3,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2016-11-19
4,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2016-11-20


# basic procedure for need data_diff

In [47]:
df['date_diff'] = [i.days for i in (df.date - df['date'].shift(1))]
print ('shape of df:', df.shape)

df = df.groupby('msno').apply(drop_first_columns) # 每個user第一欄不用
df.reset_index(drop = True, inplace = True)
df.head()

shape of df: (49604, 3)


,msno,date,date_diff
0,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2016-11-19,1.0
1,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2016-11-20,1.0
2,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2016-11-21,1.0
3,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2016-11-22,1.0
4,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2016-11-23,1.0


In [39]:
T = 0
folder = 'trainW-'+ str(T)

# num_in_a_row

In [10]:
T = 0
folder = 'trainW-'+ str(T)
##################################################
# All history
##################################################

# core
tbl = df[df.date_diff == 1].groupby('msno').date_diff.size().to_frame() # date_diff == 1: mean in a row
tbl.columns = ['listen_music_in_a_row_count']
tbl['listen_music_in_a_row_ratio'] = tbl.listen_music_in_a_row_count / df.groupby('msno').date_diff.apply(len)
tbl.reset_index(inplace = True)
#==============================================================================
print('reduce memory')
#==============================================================================
utils.reduce_memory(tbl)
# write
tbl.to_csv('../../feature/{}/listen_music_in_a_row_count.csv'.format(folder), index = False)
del tbl
gc.collect()
##################################################
# n = 7
##################################################
df_ = df.groupby('msno').apply(within_n_days, T, n = 7).reset_index(drop = True)
#core
tbl = df_[df_.date_diff == 1].groupby('msno').date_diff.size().to_frame()
tbl.columns = ['listen_music_in_a_row_count_during_t_7']
tbl['listen_music_in_a_row_ratio_during_t_7'] = tbl.listen_music_in_a_row_count_during_t_7 / df_.groupby('msno').date_diff.apply(len)
tbl.reset_index(inplace = True)
del df_
gc.collect()
#==============================================================================
print('reduce memory')
#==============================================================================
utils.reduce_memory(tbl)
# write
tbl.to_csv('../../feature/{}/listen_music_in_a_row_count_during_t_7.csv'.format(folder), index = False)
del tbl
gc.collect()
##################################################
# n = 14
##################################################
df_ = df.groupby('msno').apply(within_n_days, T, n = 14).reset_index(drop = True)
#core
tbl = df_[df_.date_diff == 1].groupby('msno').date_diff.size().to_frame()
tbl.columns = ['listen_music_in_a_row_count_during_t_14']
tbl['listen_music_in_a_row_ratio_during_t_14'] = tbl.listen_music_in_a_row_count_during_t_14 / df_.groupby('msno').date_diff.apply(len)
tbl.reset_index(inplace = True)
del df_
gc.collect()
#==============================================================================
print('reduce memory')
#==============================================================================
utils.reduce_memory(tbl)
# write
tbl.to_csv('../../feature/{}/listen_music_in_a_row_count_during_t_14.csv'.format(folder), index = False)
del tbl
gc.collect()
##################################################
# n = 30
##################################################
df_ = df.groupby('msno').apply(within_n_days, T, n = 30).reset_index(drop = True)
#core
tbl = df_[df_.date_diff == 1].groupby('msno').date_diff.size().to_frame()
tbl.columns = ['listen_music_in_a_row_count_during_t_30']
tbl['listen_music_in_a_row_ratio_during_t_30'] = tbl.listen_music_in_a_row_count_during_t_30 / df_.groupby('msno').date_diff.apply(len)
tbl.reset_index(inplace = True)
del df_
gc.collect()
#==============================================================================
print('reduce memory')
#==============================================================================
utils.reduce_memory(tbl)
# write
tbl.to_csv('../../feature/{}/listen_music_in_a_row_count_during_t_30.csv'.format(folder), index = False)
del tbl
gc.collect()
##################################################
# n = 60
##################################################
df_ = df.groupby('msno').apply(within_n_days, T, n = 60).reset_index(drop = True)
#core
tbl = df_[df_.date_diff == 1].groupby('msno').date_diff.size().to_frame()
tbl.columns = ['listen_music_in_a_row_count_during_t_60']
tbl['listen_music_in_a_row_ratio_during_t_60'] = tbl.listen_music_in_a_row_count_during_t_60 / df_.groupby('msno').date_diff.apply(len)
tbl.reset_index(inplace = True)
del df_
gc.collect()
#==============================================================================
print('reduce memory')
#==============================================================================
utils.reduce_memory(tbl)
# write
tbl.to_csv('../../feature/{}/listen_music_in_a_row_count_during_t_60.csv'.format(folder), index = False)
del tbl
gc.collect()
##################################################
# n = 90
##################################################
df_ = df.groupby('msno').apply(within_n_days, T, n = 90).reset_index(drop = True)
#core
tbl = df_[df_.date_diff == 1].groupby('msno').date_diff.size().to_frame()
tbl.columns = ['listen_music_in_a_row_count_during_t_90']
tbl['listen_music_in_a_row_ratio_during_t_90'] = tbl.listen_music_in_a_row_count_during_t_90 / df_.groupby('msno').date_diff.apply(len)
tbl.reset_index(inplace = True)
del df_
gc.collect()
#==============================================================================
print('reduce memory')
#==============================================================================
utils.reduce_memory(tbl)
# write
tbl.to_csv('../../feature/{}/listen_music_in_a_row_count_during_t_90.csv'.format(folder), index = False)
del tbl
gc.collect()

100%|██████████| 3/3 [00:00<00:00, 1366.22it/s]

reduce memory



100%|██████████| 3/3 [00:00<00:00, 1639.47it/s]

reduce memory



100%|██████████| 3/3 [00:00<00:00, 1482.26it/s]

...


100%|██████████| 3/3 [00:00<00:00, 1300.42it/s]

reduce memory



100%|██████████| 3/3 [00:00<00:00, 1425.02it/s]

reduce memory


27

# date_diff

In [11]:
df.head()

,msno,date,date_diff
0,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2016-11-17,1.0
1,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2016-11-18,1.0
2,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2016-11-19,1.0
3,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2016-11-20,1.0
4,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2016-11-21,1.0


In [12]:
##################################################
# All history
##################################################

#core
tbl = df.groupby('msno').date_diff.mean().to_frame()
tbl.columns = ['date_diff-mean']
tbl['date_diff-min'] = df.groupby('msno').date_diff.min()
tbl['date_diff-max'] = df.groupby('msno').date_diff.max()
tbl['date_diff-median'] = df.groupby('msno').date_diff.median()
tbl['date_diff-std'] = df.groupby('msno').date_diff.std()
tbl.reset_index(inplace = True)
#==============================================================================
print('reduce memory')
#==============================================================================
utils.reduce_memory(tbl)
# write
tbl.to_csv('../../feature/{}/date_diff.csv'.format(folder), index = False)
del tbl
gc.collect()

##################################################
# n = 7
##################################################
df_ = df.groupby('msno').apply(within_n_days,T, n = 7).reset_index(drop = True)
#core
tbl = df_.groupby('msno').date_diff.mean().to_frame()
tbl.columns = ['date_diff_during_t_7-mean']
tbl['date_diff_during_t_7-min'] = df_.groupby('msno').date_diff.min()
tbl['date_diff_during_t_7-max'] = df_.groupby('msno').date_diff.max()
tbl['date_diff_during_t_7-median'] = df_.groupby('msno').date_diff.median()
tbl['date_diff_during_t_7-std'] = df_.groupby('msno').date_diff.std()
tbl.reset_index(inplace = True)
del df_
gc.collect()
#==============================================================================
print('reduce memory')
#==============================================================================
utils.reduce_memory(tbl)
# write
tbl.to_csv('../../feature/{}/date_diff_during_t_7.csv'.format(folder), index = False)
del tbl
gc.collect()

##################################################
# n = 14
##################################################
df_ = df.groupby('msno').apply(within_n_days,T, n = 14).reset_index(drop = True)
#core
tbl = df_.groupby('msno').date_diff.mean().to_frame()
tbl.columns = ['date_diff_during_t_14-mean']
tbl['date_diff_during_t_14-min'] = df_.groupby('msno').date_diff.min()
tbl['date_diff_during_t_14-max'] = df_.groupby('msno').date_diff.max()
tbl['date_diff_during_t_14-median'] = df_.groupby('msno').date_diff.median()
tbl['date_diff_during_t_14-std'] = df_.groupby('msno').date_diff.std()
tbl.reset_index(inplace = True)
del df_
gc.collect()
#==============================================================================
print('reduce memory')
#==============================================================================
utils.reduce_memory(tbl)
# write
tbl.to_csv('../../feature/{}/date_diff_during_t_14.csv'.format(folder), index = False)
del tbl
gc.collect()

##################################################
# n = 30
##################################################
df_ = df.groupby('msno').apply(within_n_days,T, n = 30).reset_index(drop = True)	
#core
tbl = df_.groupby('msno').date_diff.mean().to_frame()
tbl.columns = ['date_diff_during_t_30-mean']
tbl['date_diff_during_t_30-min'] = df_.groupby('msno').date_diff.min()
tbl['date_diff_during_t_30-max'] = df_.groupby('msno').date_diff.max()
tbl['date_diff_during_t_30-median'] = df_.groupby('msno').date_diff.median()
tbl['date_diff_during_t_30-std'] = df_.groupby('msno').date_diff.std()
tbl.reset_index(inplace = True)
del df_
gc.collect()
#==============================================================================
print('reduce memory')
#==============================================================================
utils.reduce_memory(tbl)
# write
tbl.to_csv('../../feature/{}/date_diff_during_t_30.csv'.format(folder), index = False)
del tbl
gc.collect()

##################################################
# n = 60
##################################################
df_ = df.groupby('msno').apply(within_n_days,T, n = 60).reset_index(drop = True)
#core
tbl = df_.groupby('msno').date_diff.mean().to_frame()
tbl.columns = ['date_diff_during_t_60-mean']
tbl['date_diff_during_t_60-min'] = df_.groupby('msno').date_diff.min()
tbl['date_diff_during_t_60-max'] = df_.groupby('msno').date_diff.max()
tbl['date_diff_during_t_60-median'] = df_.groupby('msno').date_diff.median()
tbl['date_diff_during_t_60-std'] = df_.groupby('msno').date_diff.std()
tbl.reset_index(inplace = True)
del df_
gc.collect()
#==============================================================================
print('reduce memory')
#==============================================================================
utils.reduce_memory(tbl)
# write
tbl.to_csv('../../feature/{}/date_diff_during_t_60.csv'.format(folder), index = False)
del tbl
gc.collect()
##################################################
# n = 90
##################################################
df_ = df.groupby('msno').apply(within_n_days,T, n = 90).reset_index(drop = True)
#core
tbl = df_.groupby('msno').date_diff.mean().to_frame()
tbl.columns = ['date_diff_during_t_90-mean']
tbl['date_diff_during_t_90-min'] = df_.groupby('msno').date_diff.min()
tbl['date_diff_during_t_90-max'] = df_.groupby('msno').date_diff.max()
tbl['date_diff_during_t_90-median'] = df_.groupby('msno').date_diff.median()
tbl['date_diff_during_t_90-std'] = df_.groupby('msno').date_diff.std()
tbl.reset_index(inplace = True)
del df_
gc.collect()
#==============================================================================
print('reduce memory')
#==============================================================================
utils.reduce_memory(tbl)
# write
tbl.to_csv('../../feature/{}/date_diff_during_t_90.csv'.format(folder), index = False)
del tbl
gc.collect()

100%|██████████| 6/6 [00:00<00:00, 1141.93it/s]

reduce memory



100%|██████████| 6/6 [00:00<00:00, 1057.83it/s]

reduce memory



100%|██████████| 6/6 [00:00<00:00, 1868.01it/s]


...


100%|██████████| 6/6 [00:00<00:00, 1331.53it/s]

reduce memory



100%|██████████| 6/6 [00:00<00:00, 1374.06it/s]

reduce memory


36

# num_log_in

In [70]:
df = pd.read_csv('small_df_for_developing_and_debugging.csv', parse_dates = ['date'])
df.head()

,msno,date
0,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2016-11-16
1,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2016-11-17
2,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2016-11-18
3,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2016-11-19
4,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2016-11-20


In [71]:
def listening_longevity(x):
    # input_col = ['msno','date']
    x['listening_longevity'] = (x.iloc[-1].date - x.iloc[0].date).days
    return x


In [72]:
##################################################
# All history
##################################################
# count
tbl = df.groupby('msno').date.size().to_frame()
tbl.columns = ['num_log_in']
tbl.reset_index(inplace = True)
# for computing log_in_ratio
user_logs_copy = df.groupby('msno').apply(listening_longevity)
user_logs_copy.drop_duplicates('msno',inplace =True)

tbl = pd.merge(tbl,user_logs_copy, on = 'msno', how = 'left')

del user_logs_copy
gc.collect()   
#log_in_ratio
tbl['log_in_ratio'] = tbl.num_log_in/tbl.listening_longevity
tbl.drop(['date','listening_longevity'], axis = 1,inplace = True)
# write
tbl.to_csv('../../feature/{}/num_log_in.csv'.format(folder), index = False)

del tbl
gc.collect()
##################################################
# n = 7
##################################################
df_ = df.groupby('msno').apply(within_n_days,T, n = 7).reset_index(drop = True)   
tbl = df_.groupby('msno').date.size().to_frame()
tbl.columns = ['num_log_in_during_t_7']
tbl.reset_index(inplace = True)
# for computing log_in_ratio
user_logs_copy = df_.groupby('msno').apply(listening_longevity)
user_logs_copy.drop_duplicates('msno',inplace =True)

tbl = pd.merge(tbl,user_logs_copy, on = 'msno', how = 'left')

del user_logs_copy

gc.collect()   
#log_in_ratio
tbl['log_in_ratio_during_t_7'] = tbl.num_log_in_during_t_7/tbl.listening_longevity
tbl.drop(['date','listening_longevity'], axis = 1, inplace = True)
# write
tbl.to_csv('../../feature/{}/num_log_in_during_t_7.csv'.format(folder), index = False)

del tbl
gc.collect()

##################################################
# n = 14
##################################################
df_ = df.groupby('msno').apply(within_n_days,T, n = 14).reset_index(drop = True)   
tbl = df_.groupby('msno').date.size().to_frame()
tbl.columns = ['num_log_in_during_t_14']
tbl.reset_index(inplace = True)
# for computing log_in_ratio
user_logs_copy = df_.groupby('msno').apply(listening_longevity)
user_logs_copy.drop_duplicates('msno',inplace =True)

tbl = pd.merge(tbl,user_logs_copy, on = 'msno', how = 'left')

del user_logs_copy
#==============================================================================
print('reduce memory')
#==============================================================================
utils.reduce_memory(tbl)
gc.collect()   
#log_in_ratio
tbl['log_in_ratio_during_t_14'] = tbl.num_log_in_during_t_14/tbl.listening_longevity
tbl.drop(['date','listening_longevity'], axis = 1, inplace = True)
# write
tbl.to_csv('../../feature/{}/num_log_in_during_t_14.csv'.format(folder), index = False)

del tbl
gc.collect()


##################################################
# n = 30
##################################################
df_ = df.groupby('msno').apply(within_n_days,T, n = 30).reset_index(drop = True)   
tbl = df_.groupby('msno').date.size().to_frame()
tbl.columns = ['num_log_in_during_t_30']
tbl.reset_index(inplace = True)
# for computing log_in_ratio
user_logs_copy = df_.groupby('msno').apply(listening_longevity)
user_logs_copy.drop_duplicates('msno',inplace =True)

tbl = pd.merge(tbl,user_logs_copy, on = 'msno', how = 'left')

del user_logs_copy
gc.collect()   
#log_in_ratio
tbl['log_in_ratio_during_t_30'] = tbl.num_log_in_during_t_30/tbl.listening_longevity
tbl.drop(['date','listening_longevity'], axis = 1, inplace = True)
# write
tbl.to_csv('../../feature/{}/num_log_in_during_t_30.csv'.format(folder), index = False)

del tbl
gc.collect()
##################################################
# n = 60
##################################################
df_ = df.groupby('msno').apply(within_n_days,T, n = 60).reset_index(drop = True)   
tbl = df_.groupby('msno').date.size().to_frame()
tbl.columns = ['num_log_in_during_t_60']
tbl.reset_index(inplace = True)
# for computing log_in_ratio
user_logs_copy = df_.groupby('msno').apply(listening_longevity)
user_logs_copy.drop_duplicates('msno',inplace =True)

tbl = pd.merge(tbl,user_logs_copy, on = 'msno', how = 'left')

del user_logs_copy
gc.collect()   
#log_in_ratio
tbl['log_in_ratio_during_t_60'] = tbl.num_log_in_during_t_60/tbl.listening_longevity
tbl.drop(['date','listening_longevity'], axis = 1, inplace = True)
# write
tbl.to_csv('../../feature/{}/num_log_in_during_t_60.csv'.format(folder), index = False)

del tbl
gc.collect()

##################################################
# n = 90
##################################################
df_ = df.groupby('msno').apply(within_n_days,T, n = 90).reset_index(drop = True)   
tbl = df_.groupby('msno').date.size().to_frame()
tbl.columns = ['num_log_in_during_t_90']
tbl.reset_index(inplace = True)
# for computing log_in_ratio
user_logs_copy = df_.groupby('msno').apply(listening_longevity)
user_logs_copy.drop_duplicates('msno',inplace =True)

tbl = pd.merge(tbl,user_logs_copy, on = 'msno', how = 'left')

del user_logs_copy
gc.collect()   
#log_in_ratio
tbl['log_in_ratio_during_t_90'] = tbl.num_log_in_during_t_90/tbl.listening_longevity
tbl.drop(['date','listening_longevity'], axis = 1, inplace = True)
# write
tbl.to_csv('../../feature/{}/num_log_in_during_t_90.csv'.format(folder), index = False)

del tbl
gc.collect()

100%|██████████| 4/4 [00:00<00:00, 2992.19it/s]

reduce memory


14

In [73]:
##################################################
# All history
##################################################
# count
tbl = df.groupby('msno').date.size().to_frame()
tbl.columns = ['num_log_in']
tbl.reset_index(inplace = True)
tbl['log_in_ratio'] = tbl.num_log_in/ df.groupby('msno').apply(listening_longevity).listening_longevity

In [74]:
tbl.head()

,msno,num_log_in,log_in_ratio
0,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,103,0.990385
1,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,609,5.855769
2,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,603,5.798077
3,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,292,2.807692
4,++/UDNo9DLrxT8QVGiDi1OnWfczAdEwThaVyD0fXO50=,445,4.278846


In [75]:
##################################################
# All history
##################################################
# count
tbl = df.groupby('msno').date.size().to_frame()
tbl.columns = ['num_log_in']
tbl.reset_index(inplace = True)
# for computing log_in_ratio
user_logs_copy = df.groupby('msno').apply(listening_longevity)
user_logs_copy.drop_duplicates('msno',inplace =True)

tbl = pd.merge(tbl,user_logs_copy, on = 'msno', how = 'left')

del user_logs_copy
gc.collect()   
#log_in_ratio
tbl['log_in_ratio'] = tbl.num_log_in/tbl.listening_longevity
tbl.drop(['date','listening_longevity'], axis = 1,inplace = True)


In [76]:
tbl.head()

,msno,num_log_in,log_in_ratio
0,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,103,0.990385
1,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,609,0.772843
2,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,603,0.764259
3,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,292,0.839080
4,++/UDNo9DLrxT8QVGiDi1OnWfczAdEwThaVyD0fXO50=,445,0.567602


In [66]:
user_logs_copy = df.groupby('msno').apply(listening_longevity)


In [67]:
user_logs_copy

,msno,date,date_diff,listening_longevity
0,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2016-11-19,1.0,101
1,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2016-11-20,1.0,101
2,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2016-11-21,1.0,101
3,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2016-11-22,1.0,101
4,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2016-11-23,1.0,101
5,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2016-11-24,1.0,101
6,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2016-11-25,1.0,101
7,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2016-11-26,1.0,101
8,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2016-11-27,1.0,101
9,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2016-11-28,1.0,101
